# 🎓 Week 10 - Day 1: Computer Vision & CNN Basics
## Visual Journey into Convolutional Neural Networks

### Today's Learning Goals:
- ✅ Understand how images are represented as numbers
- ✅ Visualize RGB channels and pixel grids
- ✅ Master the convolution operation step-by-step
- ✅ Apply different kernels (edge detection, blur, sharpen)
- ✅ Experiment with padding and stride effects
- ✅ Calculate output dimensions
- ✅ Visualize feature maps from real CNNs
- ✅ Build your first convolutional layer in PyTorch

---

**Let's make computer vision intuitive with lots of visualizations!** 🖼️

## 📦 Part 1: Setup and Image Loading

First, let's import all libraries and load a sample image to work with!

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("✅ All libraries imported successfully!")
print(f"✅ Using PyTorch version: {torch.__version__}")

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\Users\Zigron\anaconda3\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [ ]:
# Load a sample image
# Using a simple cat image from the internet
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Resize to manageable size
img = img.resize((224, 224))

# Convert to numpy array
img_array = np.array(img)

print(f"✅ Image loaded successfully!")
print(f"📊 Image shape: {img_array.shape}")
print(f"📊 Data type: {img_array.dtype}")
print(f"📊 Pixel value range: [{img_array.min()}, {img_array.max()}]")

# Display the image
plt.figure(figsize=(6, 6))
plt.imshow(img_array)
plt.title("Our Sample Image", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n💡 This image is a 3D array: (Height=224, Width=224, Channels=3)")

## 🎨 Part 2: Understanding Image Representation

Let's visualize how computers see images - as grids of numbers!

In [ ]:
# Create a simple 8x8 grayscale image to understand pixel values
simple_img = np.array([
    [0, 0, 0, 255, 255, 0, 0, 0],
    [0, 0, 255, 255, 255, 255, 0, 0],
    [0, 255, 255, 255, 255, 255, 255, 0],
    [255, 255, 0, 255, 255, 0, 255, 255],
    [255, 255, 255, 255, 255, 255, 255, 255],
    [255, 0, 255, 255, 255, 255, 0, 255],
    [0, 0, 0, 255, 255, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.uint8)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Image visualization
axes[0].imshow(simple_img, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Grayscale Image (Visual)", fontsize=14, fontweight='bold')
axes[0].axis('off')

# Number grid visualization
axes[1].imshow(simple_img, cmap='gray', vmin=0, vmax=255)
axes[1].set_title("Same Image (With Pixel Values)", fontsize=14, fontweight='bold')
for i in range(8):
    for j in range(8):
        color = 'black' if simple_img[i, j] > 128 else 'white'
        axes[1].text(j, i, str(simple_img[i, j]), 
                   ha='center', va='center', color=color, fontsize=9, fontweight='bold')
axes[1].set_xticks(range(8))
axes[1].set_yticks(range(8))
axes[1].grid(True, color='red', linewidth=1.5)

plt.tight_layout()
plt.show()

print("\n🎯 Key Insights:")
print("• 0 = Black pixel")
print("• 255 = White pixel")
print("• Numbers in between = Shades of gray")
print("• Computers see this 8×8 grid as 64 numbers!")

In [ ]:
# Visualize RGB channels separately
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Original image
axes[0, 0].imshow(img_array)
axes[0, 0].set_title("Original RGB Image", fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Red channel
red_channel = img_array[:, :, 0]
axes[0, 1].imshow(red_channel, cmap='Reds')
axes[0, 1].set_title(f"Red Channel (Shape: {red_channel.shape})", fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# Green channel
green_channel = img_array[:, :, 1]
axes[1, 0].imshow(green_channel, cmap='Greens')
axes[1, 0].set_title(f"Green Channel (Shape: {green_channel.shape})", fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

# Blue channel
blue_channel = img_array[:, :, 2]
axes[1, 1].imshow(blue_channel, cmap='Blues')
axes[1, 1].set_title(f"Blue Channel (Shape: {blue_channel.shape})", fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n🎨 RGB Channel Breakdown:")
print(f"• Original image: {img_array.shape} = (Height × Width × Channels)")
print(f"• Each channel: {red_channel.shape} = (Height × Width)")
print(f"• Total numbers: {img_array.size:,} values!")
print(f"\n💡 The computer processes {img_array.size:,} numbers for this one image!")

## 🔍 Part 3: Manual Convolution Operation

Let's perform convolution step-by-step to understand exactly how it works!

In [ ]:
def manual_convolution(image, kernel, stride=1, padding=0):
    """
    Perform 2D convolution manually (step-by-step)
    
    Args:
        image: Input 2D array
        kernel: Filter/kernel 2D array
        stride: Step size for sliding window
        padding: Padding around the image
    
    Returns:
        Output feature map
    """
    # Add padding if needed
    if padding > 0:
        image = np.pad(image, padding, mode='constant', constant_values=0)
    
    # Get dimensions
    img_h, img_w = image.shape
    kernel_h, kernel_w = kernel.shape
    
    # Calculate output dimensions
    out_h = (img_h - kernel_h) // stride + 1
    out_w = (img_w - kernel_w) // stride + 1
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Perform convolution
    for i in range(out_h):
        for j in range(out_w):
            # Extract region
            h_start = i * stride
            w_start = j * stride
            region = image[h_start:h_start+kernel_h, w_start:w_start+kernel_w]
            
            # Element-wise multiplication and sum
            output[i, j] = np.sum(region * kernel)
    
    return output

# Create a simple 5x5 input
input_img = np.array([
    [1, 2, 3, 0, 1],
    [0, 1, 2, 3, 0],
    [3, 0, 1, 2, 3],
    [2, 3, 0, 1, 0],
    [1, 0, 3, 2, 1]
], dtype=np.float32)

# Create a 3x3 edge detection kernel (vertical)
edge_kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
], dtype=np.float32)

# Apply convolution
output = manual_convolution(input_img, edge_kernel, stride=1, padding=0)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Input
im1 = axes[0].imshow(input_img, cmap='viridis', vmin=0, vmax=3)
axes[0].set_title("Input Image (5×5)", fontsize=14, fontweight='bold')
for i in range(5):
    for j in range(5):
        axes[0].text(j, i, f'{input_img[i,j]:.0f}', 
                   ha='center', va='center', color='white', fontsize=10, fontweight='bold')
axes[0].set_xticks(range(5))
axes[0].set_yticks(range(5))
axes[0].grid(True, color='white', linewidth=1)
plt.colorbar(im1, ax=axes[0])

# Kernel
im2 = axes[1].imshow(edge_kernel, cmap='coolwarm', vmin=-1, vmax=1)
axes[1].set_title("Vertical Edge Kernel (3×3)", fontsize=14, fontweight='bold')
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{edge_kernel[i,j]:.0f}', 
                   ha='center', va='center', color='black', fontsize=12, fontweight='bold')
axes[1].set_xticks(range(3))
axes[1].set_yticks(range(3))
axes[1].grid(True, color='black', linewidth=1.5)
plt.colorbar(im2, ax=axes[1])

# Output
im3 = axes[2].imshow(output, cmap='plasma')
axes[2].set_title(f"Output Feature Map ({output.shape[0]}×{output.shape[1]})", 
                 fontsize=14, fontweight='bold')
for i in range(output.shape[0]):
    for j in range(output.shape[1]):
        axes[2].text(j, i, f'{output[i,j]:.0f}', 
                   ha='center', va='center', color='white', fontsize=10, fontweight='bold')
axes[2].set_xticks(range(output.shape[1]))
axes[2].set_yticks(range(output.shape[0]))
axes[2].grid(True, color='white', linewidth=1)
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print("\n🎯 Convolution Summary:")
print(f"• Input size: {input_img.shape}")
print(f"• Kernel size: {edge_kernel.shape}")
print(f"• Output size: {output.shape}")
print(f"• Stride: 1, Padding: 0")
print(f"\n💡 Notice how the output is smaller: 5×5 → 3×3")

## 🎯 Challenge - Build Your Own CNN Layer

**Your Task:** Create a convolutional layer that:
1. Takes a grayscale image (1 channel)
2. Outputs 8 feature maps
3. Uses 5×5 kernel
4. Maintains input size (use appropriate padding)
5. Visualize the output feature maps

**Hints:**
- For same padding with 5×5 kernel: padding = 2
- Convert to grayscale: gray_img = torch.from_numpy(np.mean(img_array, axis=2))


In [ ]:
# YOUR CODE HERE


## 📚 Summary & Key Takeaways

### 🎓 What We Learned Today:

**1. Image Representation:** Images are grids of numbers (0-255)

**2. Convolution:** Sliding window that detects patterns

**3. Kernels:** Small matrices that detect specific features

**4. Padding:** Preserves spatial dimensions

**5. Stride:** Controls output size

**6. Feature Maps:** Shows where patterns detected

**7. Receptive Field:** Grows with network depth

### 💡 Pro Tips:
- Use 3×3 kernels (most common)
- Same padding preserves size
- Stride=2 for downsampling
- Visualize to understand!

**Great job! You've mastered CNN fundamentals! 🎉**